## MISSION TO MARS SCRAPER

Web Scraping Activity - USC Data Science BootCamp

## Part 1 = Nasa Mars News

In [ ]:
# Import Modules
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pymongo

In [ ]:
# declaring element names to grab
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
article_header = 'slide'
content_title = 'content_title'
content_body = 'article_teaser_body'
wait_element = 'news'
driver = webdriver.Chrome('/Users/Justin/Desktop/Web-Scraping/chromedriver.exe')
# get url and wait until page is fully loaded before proceeding
driver.get(url)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID,wait_element))
    )
except:
    pass

In [ ]:
#grab first slide
slide = driver.find_elements_by_class_name(article_header)[0]

In [ ]:
#extract text of first title and first paragraph
first_article_headline = slide.find_element_by_class_name(content_title).text
first_article_paragraph = slide.find_element_by_class_name(content_body).text

## Part 2 - JPL Mars Featured Image

In [ ]:
#JPL featured image URL and request 
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
featured_image_response = requests.get(featured_image_url)

if featured_image_response.status_code == 200:
    featured_image_html = featured_image_response.text

In [ ]:
#Beautiful soup to parse html and locate desired image url
featured_image_soup = bs(featured_image_html, "html.parser")
image_element = featured_image_soup.find("a", class_="button fancybox").get('data-fancybox-href')
image_element

In [ ]:
#full image url
full_image_link = "https://www.jpl.nasa.gov" + image_element
full_image_link

## Part 3 - Mars Weather

In [ ]:
#Weather twitter url
weather_tweets_url = "https://twitter.com/marswxreport?lang=en"
driver.get(weather_tweets_url)

In [ ]:
#Search by xpath
mars_weather = driver.find_element_by_xpath("/html/body/div/div/div/div[2]/main/div/div/div/div/div/div/div/div/div[2]/section/div/div/div/div[1]/div/div/div/article/div/div[2]/div[2]/div[2]/div[1]/div/span")
mars_weather.text

## Part 4 - Mars Facts

In [ ]:
#Mars facts URL
mars_facts_url = "https://space-facts.com/mars/"

In [ ]:
#Extract first table and reset index
all_mars_facts_tables = pd.read_html(mars_facts_url)
mars_facts_df = all_mars_facts_tables[1].set_index(0).T
mars_facts_df

## Part 5 - Hemisphere Images

In [ ]:
#Mars Hemispheres URL
mars_hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
mars_hemisphere_response = requests.get(mars_hemisphere_url)

if mars_hemisphere_response.status_code == 200:
    mars_hemisphere_html = mars_hemisphere_response.text

In [ ]:
#Define function to grab image url from each hemisphere link
def get_full_image_link(link):
    response = requests.get(link)
    if response.status_code == 200:
        html = response.text
    soup = bs(html, "html.parser")
    img_link = soup.find("a", text="Original").get("href")
    title = soup.find("h2", class_="title").text.split(" Enhanced")[0]
    return [title, img_link]

In [ ]:
#Beautiful soup to parse through HTML using above function
mars_hemisphere_soup = bs(mars_hemisphere_html, "html.parser")
link_elements = mars_hemisphere_soup.findAll("a", {"class": "itemLink product-item"})
full_page_links = [dict(title=get_full_image_link("https://astrogeology.usgs.gov" + le.get('href'))[0], img_url=get_full_image_link("https://astrogeology.usgs.gov" + le.get('href'))[1]) for le in link_elements]
full_page_links